## Advent of Code 2024 - Day 15

In [1]:
from rich import print
from httpx import request
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from itertools import product
import math
import functools

%load_ext rich

In [206]:
def parse_input(path):
    # Read file and split into lines
    with open(path, "r") as file:
        result = file.read().split("\n\n")
    # Optional: Remove any empty lines if needed
    return [result[0].split("\n"), result[1]]

In [207]:
sample_input = parse_input("sample.txt")
sample_input_2 = parse_input("sample_2.txt")
actual_input = parse_input("input.txt")

## Part 1

In [208]:
floor_map, moves_list = sample_input_2
floor_map, moves_list


(
    ['########', '#..O.O.#', '##..O@.#', '#...O..#', '#.#.#..#', '#...O..#', '#......#', '########'],
    '<^^>>>vv<v>>v<<'
)

In [209]:
np.array(list(map(list, floor_map)))


array([['#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '.', '.', 'O', '.', 'O', '.', '#'],
       ['#', '#', '.', '.', 'O', '@', '.', '#'],
       ['#', '.', '.', '.', 'O', '.', '.', '#'],
       ['#', '.', '#', '.', '#', '.', '.', '#'],
       ['#', '.', '.', '.', 'O', '.', '.', '#'],
       ['#', '.', '.', '.', '.', '.', '.', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [210]:
class Floor:
    def __init__(self, floor):
        self.floor = np.array(list(map(list, floor)))

        self.robot_pos = np.where(self.floor == "@")
        self.robot_pos = (int(self.robot_pos[0][0]), int(self.robot_pos[1][0]))

        self.directions = {
            ">": (0, 1),
            "<": (0, -1),
            "^": (-1, 0),
            "v": (1, 0),
        }

    def move(self, direction):
        next_pos = tuple(map(sum, zip(self.robot_pos, self.directions[direction])))

        if self.floor[next_pos] == "#":
            return None

        if self.floor[next_pos] == ".":
            self.floor[self.robot_pos] = "."
            self.floor[next_pos] = "@"

            self.robot_pos = next_pos

            return None

        if self.floor[next_pos] == "O":
            print(f"Next position: {next_pos}, found O")

            if direction == ">":
                row_vals = self.floor[next_pos[0], next_pos[1] :]
                empty_spaces = np.where(row_vals == ".")[0]

                if len(empty_spaces) == 0:
                    return None

                next_empty_space = (
                    next_pos[0],
                    next_pos[1] + min(empty_spaces),
                )

                print(next_empty_space)

                for i in range(next_pos[1], next_empty_space[1] + 1):
                    self.floor[next_pos[0], i] = "O"

            elif direction == "<":
                row_vals = self.floor[next_pos[0], : next_pos[1]]
                empty_spaces = np.where(row_vals == ".")[0]

                if len(empty_spaces) == 0:
                    return None

                next_empty_space = (
                    next_pos[0],
                    max(empty_spaces),
                )

                for i in range(next_pos[1], next_empty_space[1] - 1, -1):
                    self.floor[next_pos[0], i] = "O"

            elif direction == "^":
                col_vals = self.floor[: next_pos[0], next_pos[1]]
                empty_spaces = np.where(col_vals == ".")[0]

                if len(empty_spaces) == 0:
                    return None

                next_empty_space = (max(empty_spaces), next_pos[1])

                for i in range(next_pos[0], next_empty_space[0] - 1, -1):
                    self.floor[i, next_pos[1]] = "O"

            elif direction == "v":
                col_vals = self.floor[next_pos[0] :, next_pos[1]]
                empty_spaces = np.where(col_vals == ".")[0]

                print(col_vals, empty_spaces)

                if len(empty_spaces) == 0:
                    return None

                next_empty_space = (next_pos[0] + min(empty_spaces), next_pos[1])
                print(next_empty_space)

                for i in range(next_pos[0], next_empty_space[0] + 1):
                    print(f"operating on {i, next_pos[1]}")
                    self.floor[i, next_pos[1]] = "O"

            self.floor[self.robot_pos] = "."
            self.floor[next_pos] = "@"

            self.robot_pos = next_pos

            return None


In [211]:
floor = Floor(floor_map)

In [212]:
floor.floor


array([['#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '.', '.', 'O', '.', 'O', '.', '#'],
       ['#', '#', '.', '.', 'O', '@', '.', '#'],
       ['#', '.', '.', '.', 'O', '.', '.', '#'],
       ['#', '.', '#', '.', '#', '.', '.', '#'],
       ['#', '.', '.', '.', 'O', '.', '.', '#'],
       ['#', '.', '.', '.', '.', '.', '.', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [213]:
# floor.move("^")
# floor.move(">")
# floor.move(">")
# floor.move(">")
floor.move("<")
floor.move("v")
# floor.move("<")
# floor.move("<")

Next position: (2, 4), found O

Next position: (3, 4), found O

['O' '#' 'O' '.' '#'] [3]

(np.int64(6), 4)

operating on (3, 4)

operating on (4, 4)

operating on (5, 4)

operating on (6, 4)

In [214]:
floor.floor


array([['#', '#', '#', '#', '#', '#', '#', '#'],
       ['#', '.', '.', 'O', '.', 'O', '.', '#'],
       ['#', '#', '.', 'O', '.', '.', '.', '#'],
       ['#', '.', '.', '.', '@', '.', '.', '#'],
       ['#', '.', '#', '.', 'O', '.', '.', '#'],
       ['#', '.', '.', '.', 'O', '.', '.', '#'],
       ['#', '.', '.', '.', 'O', '.', '.', '#'],
       ['#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [94]:
def solution_1(input, MAX_X, MAX_Y):
    robots = [Robot(*parse_p_v(line), MAX_X, MAX_Y) for line in input]
    floor = Floor(MAX_X, MAX_Y)
    for robot in robots:
        floor.add_robot(robot)

    for robot in robots:
        robot.take_steps(100)

    floor_grid = floor.check_grid()

    q1 = [floor_grid[i][: MAX_X // 2] for i in range(MAX_Y // 2)]
    q2 = [floor_grid[i][(MAX_X // 2) + 1 :] for i in range(MAX_Y // 2)]
    q3 = [floor_grid[i][: MAX_X // 2] for i in range((MAX_Y // 2) + 1, MAX_Y)]
    q4 = [floor_grid[i][(MAX_X // 2) + 1 :] for i in range((MAX_Y // 2) + 1, MAX_Y)]

    return np.sum(q1) * np.sum(q2) * np.sum(q3) * np.sum(q4)


In [95]:
print(f"Part 1 - Sample: {solution_1(sample_input, MAX_X=11, MAX_Y=7)}")
print(f"Part 1 - Actual: {solution_1(actual_input, MAX_X=101, MAX_Y=103)}")


Part 1 - Sample: 12.0

Part 1 - Actual: 230435667.0

## Part 2

In [96]:
sample_input


[
    ['p=0,4', 'v=3,-3'],
    ['p=6,3', 'v=-1,-3'],
    ['p=10,3', 'v=-1,2'],
    ['p=2,0', 'v=2,-1'],
    ['p=0,0', 'v=1,3'],
    ['p=3,0', 'v=-2,-2'],
    ['p=7,6', 'v=-1,-3'],
    ['p=3,0', 'v=-1,-2'],
    ['p=9,3', 'v=2,3'],
    ['p=7,3', 'v=-1,2'],
    ['p=2,4', 'v=2,-3'],
    ['p=9,5', 'v=-3,-3']
]

In [115]:
import torch as t

floor_images = []


def solution_2(input, MAX_X, MAX_Y):
    robots = [Robot(*parse_p_v(line), MAX_X, MAX_Y) for line in input]
    floor = Floor(MAX_X, MAX_Y)
    for robot in robots:
        floor.add_robot(robot)

    i = 0

    while True:
        for robot in robots:
            robot.take_step()

        i += 1
        floor_grid = floor.check_grid()

        floor_images.append(t.Tensor(floor_grid))

        if np.max(floor_grid) == 1:
            print(f"Found at step {i}")
            return floor

In [116]:
final_floor = solution_2(actual_input, MAX_X=101, MAX_Y=103)

Found at step 7709